In [1]:
#REQUIREMENTS

In [2]:
import pandas as pd
import sqlalchemy
import requests
from bs4 import BeautifulSoup
import re

In [3]:
path = 'data/raw/raw_data_project_m1.db'

engine = sqlalchemy.create_engine(f'sqlite:///{path}')

In [4]:
#df_country_info = pd.read_sql_query('SELECT * FROM country_info', engine)
#df_career_info = pd.read_sql_query('SELECT * FROM career_info', engine)
#df_poll_info = pd.read_sql_query('SELECT * FROM poll_info', engine)

In [5]:
df_personal_info = pd.read_sql_query('SELECT * FROM personal_info', engine)

In [6]:
df_personal_info

,uuid,age,gender,dem_has_children,age_group
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,61 years old,male,NO,40_65
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,57 years old,male,yES,40_65
2,83127080-da3d-0133-c74f-0a81e8b09a82,32 years old,male,nO,26_39
3,15626d40-db13-0133-ea5c-0a81e8b09a82,45 years old,Male,YES,40_65
4,24954a70-db98-0133-4a64-0a81e8b09a82,41 years old,Fem,yES,40_65
...,...,...,...,...,...
9644,7d1ac020-dcb4-0133-817a-0a81e8b09a82,37 years old,FeMale,nO,26_39
9645,39f989f0-db52-0133-8482-0a81e8b09a82,53 years old,Male,yES,40_65
9646,70ce4a90-d965-0133-f5e4-0a81e8b09a82,1992,male,NO,juvenile
9647,2896e440-db3c-0133-5b67-0a81e8b09a82,47 years old,male,yES,40_65


In [7]:
#Standardize all values from the gender column

df_personal_info['gender'].unique()

array(['male', 'Male', 'Fem', 'FeMale', 'female'], dtype=object)

In [8]:
df_personal_info['gender'].value_counts()

male      3796
female    2581
Male      1298
FeMale    1125
Fem        849
Name: gender, dtype: int64

In [9]:
#CLEAN GENDER COLUMN VALUES

In [10]:
#df_personal_info['gender'].replace(r'male','Male', regex=True, inplace=True)
#df_personal_info['gender'].replace(r'feMale','Female', regex=True, inplace=True)

In [11]:
df_personal_info["gender"].replace(to_replace = ['male','Fem','FeMale','female'], value = ['Male', 'Female','Female','Female'],inplace=True)

In [12]:
df_personal_info['gender'].unique()

array(['Male', 'Female'], dtype=object)

In [13]:
df_personal_info['gender'].value_counts()

Male      5094
Female    4555
Name: gender, dtype: int64

In [14]:
#Now we have only values categorized by Male and Female

In [15]:
df_personal_info

,uuid,age,gender,dem_has_children,age_group
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,61 years old,Male,NO,40_65
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,57 years old,Male,yES,40_65
2,83127080-da3d-0133-c74f-0a81e8b09a82,32 years old,Male,nO,26_39
3,15626d40-db13-0133-ea5c-0a81e8b09a82,45 years old,Male,YES,40_65
4,24954a70-db98-0133-4a64-0a81e8b09a82,41 years old,Female,yES,40_65
...,...,...,...,...,...
9644,7d1ac020-dcb4-0133-817a-0a81e8b09a82,37 years old,Female,nO,26_39
9645,39f989f0-db52-0133-8482-0a81e8b09a82,53 years old,Male,yES,40_65
9646,70ce4a90-d965-0133-f5e4-0a81e8b09a82,1992,Male,NO,juvenile
9647,2896e440-db3c-0133-5b67-0a81e8b09a82,47 years old,Male,yES,40_65


In [16]:
#FETCH COUNTRY NAMES FROM URL AND STORE IT IN A DATAFRAME

In [17]:
url = 'https://www.iban.com/country-codes'
country_codes = pd.read_html(url,header=0)[0]
country_codes

,Country,Alpha-2 code,Alpha-3 code,Numeric
0,Afghanistan,AF,AFG,4
1,Åland Islands,AX,ALA,248
2,Albania,AL,ALB,8
3,Algeria,DZ,DZA,12
4,American Samoa,AS,ASM,16
...,...,...,...,...
244,Wallis and Futuna,WF,WLF,876
245,Western Sahara,EH,ESH,732
246,Yemen,YE,YEM,887
247,Zambia,ZM,ZMB,894


In [18]:
country_codes = country_codes.drop(columns=['Alpha-3 code','Numeric'])
country_codes.rename(columns = {'Country':'country','Alpha-2 code':'country_code'}, inplace=True)

In [19]:
country_codes

,country,country_code
0,Afghanistan,AF
1,Åland Islands,AX
2,Albania,AL
3,Algeria,DZ
4,American Samoa,AS
...,...,...
244,Wallis and Futuna,WF
245,Western Sahara,EH
246,Yemen,YE
247,Zambia,ZM


In [20]:
#CLEAN NORMALIZED JOB TITLES

In [21]:
df_career_info = pd.read_sql_query('SELECT * FROM career_info', engine)

In [22]:
df_career_info['normalized_job_code'].head(10)

0                                None
1    861a9b9151e11362eb3c77ca914172d0
2                                None
3    049a3f3a2b5f85cb2971ba77ad66e10c
4    f4b2fb1aa40f661488e2782b6d57ad2f
5    27af8700f5577cec835acee2cb90a2ff
6                                None
7    c1b670eba9ccb65e7c99f7da116d5b9c
8    d71c4164c99a5f9bca773e755a72f40f
9                                None
Name: normalized_job_code, dtype: object

In [23]:
# Saco los valores únicos de esta columna para evitar hacer llamadas innecesarias a la API de jobs

In [24]:
df_career_info['normalized_job_code'].unique()

array([None, '861a9b9151e11362eb3c77ca914172d0',
       '049a3f3a2b5f85cb2971ba77ad66e10c',
       'f4b2fb1aa40f661488e2782b6d57ad2f',
       '27af8700f5577cec835acee2cb90a2ff',
       'c1b670eba9ccb65e7c99f7da116d5b9c',
       'd71c4164c99a5f9bca773e755a72f40f',
       '2ee80cf30e2483dfe0947adc0428f8c5',
       '897ef18a9321616495831a4b4b0c1bb2',
       'a003702c0cea282e17967772dd83de5e',
       '04c82eae29e1c4a30eac0710b74ab276',
       'fcaf4aac737e53ad24b9f32f08b8740f',
       '3833bd699e7afe4665b595f282da0387',
       '19e384e977796c25780d8c28b12089c4',
       '474b962c13632a04e42a546be4f11dbb',
       'dac2e921c1f887d9efc4c06fa33c1397',
       '996da29c84f5eed00da40c230f4c4078',
       'cc2b26cc436bea36bda3bdd7367c411a',
       '25fe195cbf915c0824fa6c44e6d0008c',
       '76385cb24e831b3364279b9de5641d8f',
       '16b67d7f57698455aefbc1ae1b358b50',
       'b24ece3300d2f8ab34f9553e531183d6',
       '5db2c8d6539f983e1ca3d7c5b939e190',
       '36675c937946c8a4fc5de7d0b4c850de',
     

In [25]:
# Lo guardo en una variable por si necesito utilizarla luego para recorrer la API

job_codes = df_career_info['normalized_job_code'].unique()[1:]

In [26]:
job_codes

array(['861a9b9151e11362eb3c77ca914172d0',
       '049a3f3a2b5f85cb2971ba77ad66e10c',
       'f4b2fb1aa40f661488e2782b6d57ad2f',
       '27af8700f5577cec835acee2cb90a2ff',
       'c1b670eba9ccb65e7c99f7da116d5b9c',
       'd71c4164c99a5f9bca773e755a72f40f',
       '2ee80cf30e2483dfe0947adc0428f8c5',
       '897ef18a9321616495831a4b4b0c1bb2',
       'a003702c0cea282e17967772dd83de5e',
       '04c82eae29e1c4a30eac0710b74ab276',
       'fcaf4aac737e53ad24b9f32f08b8740f',
       '3833bd699e7afe4665b595f282da0387',
       '19e384e977796c25780d8c28b12089c4',
       '474b962c13632a04e42a546be4f11dbb',
       'dac2e921c1f887d9efc4c06fa33c1397',
       '996da29c84f5eed00da40c230f4c4078',
       'cc2b26cc436bea36bda3bdd7367c411a',
       '25fe195cbf915c0824fa6c44e6d0008c',
       '76385cb24e831b3364279b9de5641d8f',
       '16b67d7f57698455aefbc1ae1b358b50',
       'b24ece3300d2f8ab34f9553e531183d6',
       '5db2c8d6539f983e1ca3d7c5b939e190',
       '36675c937946c8a4fc5de7d0b4c850de',
       '847

In [27]:
# Ejemplo para ver que funciona con uno job code

r = requests.get("http://api.dataatwork.org/v1/jobs/861a9b9151e11362eb3c77ca914172d0")
r = r.json()
r

{'uuid': '861a9b9151e11362eb3c77ca914172d0',
 'title': 'Automatic Data Processing Planner',
 'normalized_job_title': 'automatic data processing planner',
 'parent_uuid': '0148f61d4227497728ce33490843d056'}

In [28]:
# Hago un bucle para que haga las peticiones a la API de los job_codes de los que me interesa obtener su job_title

normalized_job_list = []

for i in job_codes:
    r = requests.get("http://api.dataatwork.org/v1/jobs/{}".format(i))
    normalized_job_list.append(r.json())

df_jobs = pd.DataFrame(normalized_job_list)
    

In [29]:
df_jobs

,uuid,title,normalized_job_title,parent_uuid
0,861a9b9151e11362eb3c77ca914172d0,Automatic Data Processing Planner,automatic data processing planner,0148f61d4227497728ce33490843d056
1,049a3f3a2b5f85cb2971ba77ad66e10c,Data Coordinator,data coordinator,0b9dd32a367f4562ec77b993053d1910
2,f4b2fb1aa40f661488e2782b6d57ad2f,Database Developer,database developer,b90ca4df5690002377a7b0f1f3d40781
3,27af8700f5577cec835acee2cb90a2ff,Data Entry Specialist,data entry specialist,bf88358c56bb6cbe7eabec38ae333d19
4,c1b670eba9ccb65e7c99f7da116d5b9c,Database Architect,database architect,596226d5f9ef63de41a852826d483d19
...,...,...,...,...
151,b0fa6ede410f50b82ab74f5a705fe699,Analytical Data Miner,analytical data miner,9656fa2dc39e9643c00a45858e0117d0
152,559a21f836c93876f31b60e6d10656a7,Data Analysis Assistant,data analysis assistant,325e2251e20170928426473156bd3c48
153,c1fb1a01b78373ac2153c66fa08d16dc,Data Examination Clerk,data examination clerk,da412504dd7b130414b7bbfa2acd563b
154,05bb9a333a66d6eb151e253623efe1c0,Data Entry Clerk,data entry clerk,bf88358c56bb6cbe7eabec38ae333d19


In [30]:
# Now we only grab the columns of the dataframe that we are interested to do the merge: uuid and title

df_jobs_clean = df_jobs[['uuid','title']]
df_jobs_clean.columns = ['normalized_job_code', 'title']

In [31]:
df_jobs_clean

,normalized_job_code,title
0,861a9b9151e11362eb3c77ca914172d0,Automatic Data Processing Planner
1,049a3f3a2b5f85cb2971ba77ad66e10c,Data Coordinator
2,f4b2fb1aa40f661488e2782b6d57ad2f,Database Developer
3,27af8700f5577cec835acee2cb90a2ff,Data Entry Specialist
4,c1b670eba9ccb65e7c99f7da116d5b9c,Database Architect
...,...,...
151,b0fa6ede410f50b82ab74f5a705fe699,Analytical Data Miner
152,559a21f836c93876f31b60e6d10656a7,Data Analysis Assistant
153,c1fb1a01b78373ac2153c66fa08d16dc,Data Examination Clerk
154,05bb9a333a66d6eb151e253623efe1c0,Data Entry Clerk


In [32]:
# MERGE ALL DATAFRAMES

In [33]:
# We need to display: Country, Job Title, Gender, Quantity and Percentage

# We obtain Country from the country codes URL by using Beautiful Soup, the library for web scraping 
# We obtain Job Title from the API, we need to replace the result obtained and switch the normalized_job_code values
# We obtain the Gender column by cleaning the original df_personal_info dataframe
# Quantity is an aggregate column, maybe using groupby
# Percentage is an aggregate column as well, I'm assuming a percentage from the total of no null values

In [34]:
df_country_info = pd.read_sql_query('SELECT * FROM country_info', engine)

In [66]:
df_career_info.head()

,uuid,dem_education_level,dem_full_time_job,normalized_job_code
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,no,no,None
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,high,yes,861a9b9151e11362eb3c77ca914172d0
2,83127080-da3d-0133-c74f-0a81e8b09a82,None,no,None
3,15626d40-db13-0133-ea5c-0a81e8b09a82,high,yes,049a3f3a2b5f85cb2971ba77ad66e10c
4,24954a70-db98-0133-4a64-0a81e8b09a82,high,yes,f4b2fb1aa40f661488e2782b6d57ad2f


In [35]:
df_1 = pd.merge(df_country_info, df_career_info, how = 'inner', on = 'uuid')

In [36]:
df_2 = pd.merge(df_1, df_personal_info, how = 'inner', on = 'uuid')

In [37]:
df_2 = df_2[['uuid','country_code','normalized_job_code','gender']]

In [38]:
df_2

,uuid,country_code,normalized_job_code,gender
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,AT,None,Male
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,AT,861a9b9151e11362eb3c77ca914172d0,Male
2,83127080-da3d-0133-c74f-0a81e8b09a82,AT,None,Male
3,15626d40-db13-0133-ea5c-0a81e8b09a82,AT,049a3f3a2b5f85cb2971ba77ad66e10c,Male
4,24954a70-db98-0133-4a64-0a81e8b09a82,AT,f4b2fb1aa40f661488e2782b6d57ad2f,Female
...,...,...,...,...
9644,7d1ac020-dcb4-0133-817a-0a81e8b09a82,SK,847165cfda6b1dc82ae22b967da8af2f,Female
9645,39f989f0-db52-0133-8482-0a81e8b09a82,SK,a4d5b8b38f9513825d0d94a981ebe962,Male
9646,70ce4a90-d965-0133-f5e4-0a81e8b09a82,SK,None,Male
9647,2896e440-db3c-0133-5b67-0a81e8b09a82,SK,775190277a849cba701b306a7b374c0a,Male


In [39]:
# Merge the normalized job titles

In [40]:
df_3 = pd.merge(df_2, df_jobs_clean, how = 'inner', on = 'normalized_job_code')

In [41]:
df_3

,uuid,country_code,normalized_job_code,gender,title
0,54f0f1c0-dda1-0133-a559-0a81e8b09a82,AT,861a9b9151e11362eb3c77ca914172d0,Male,Automatic Data Processing Planner
1,cf24ed60-da3f-0133-0034-0a81e8b09a82,AT,861a9b9151e11362eb3c77ca914172d0,Female,Automatic Data Processing Planner
2,9ffc7390-dba5-0133-259b-0a81e8b09a82,BE,861a9b9151e11362eb3c77ca914172d0,Male,Automatic Data Processing Planner
3,a6361230-da58-0133-cd5a-0a81e8b09a82,BE,861a9b9151e11362eb3c77ca914172d0,Male,Automatic Data Processing Planner
4,1e2b4750-d987-0133-3c7c-0a81e8b09a82,BG,861a9b9151e11362eb3c77ca914172d0,Female,Automatic Data Processing Planner
...,...,...,...,...,...
5697,1cb1aac0-d94c-0133-8baa-0a81e8b09a82,IT,4cee16550636e292b8d136486fce943b,Male,Data Capture Clerk
5698,c8c33390-da69-0133-063a-0a81e8b09a82,PL,4cee16550636e292b8d136486fce943b,Male,Data Capture Clerk
5699,d27d24d0-d9b1-0133-03d4-0a81e8b09a82,PL,4cee16550636e292b8d136486fce943b,Male,Data Capture Clerk
5700,529f3080-d99a-0133-1b7b-0a81e8b09a82,PT,4cee16550636e292b8d136486fce943b,Male,Data Capture Clerk


In [42]:
df_3.columns

Index(['uuid', 'country_code', 'normalized_job_code', 'gender', 'title'], dtype='object')

In [43]:
df_3 = df_3[['country_code','title', 'gender']]

In [44]:
df_3

,country_code,title,gender
0,AT,Automatic Data Processing Planner,Male
1,AT,Automatic Data Processing Planner,Female
2,BE,Automatic Data Processing Planner,Male
3,BE,Automatic Data Processing Planner,Male
4,BG,Automatic Data Processing Planner,Female
...,...,...,...
5697,IT,Data Capture Clerk,Male
5698,PL,Data Capture Clerk,Male
5699,PL,Data Capture Clerk,Male
5700,PT,Data Capture Clerk,Male


In [45]:
5702 + 3947

9649

In [46]:
# Los 3947 son los valores nulos de normalized_job_title y los he desechado

In [47]:
# Añado una nueva columna

In [48]:
# Ahora hago el merge con la tabla de country codes

In [49]:
df_4 = pd.merge(df_3, country_codes, how = 'inner', on = 'country_code')

In [50]:
df_4

,country_code,title,gender,country
0,AT,Automatic Data Processing Planner,Male,Austria
1,AT,Automatic Data Processing Planner,Female,Austria
2,AT,Data Coordinator,Male,Austria
3,AT,Data Coordinator,Male,Austria
4,AT,Database Developer,Female,Austria
...,...,...,...,...
5697,MT,Geographic Information Systems Data Administra...,Male,Malta
5698,MT,Internet Database Specialist,Male,Malta
5699,MT,Database Coordinator,Male,Malta
5700,MT,Database Coordinator,Female,Malta


In [51]:
df_4 = df_4[['country','title', 'gender']]

In [58]:
df_4

,country,title,gender,Quantity
0,Austria,Automatic Data Processing Planner,Male,1
1,Austria,Automatic Data Processing Planner,Female,1
2,Austria,Data Coordinator,Male,1
3,Austria,Data Coordinator,Male,1
4,Austria,Database Developer,Female,1
...,...,...,...,...
5697,Malta,Geographic Information Systems Data Administra...,Male,1
5698,Malta,Internet Database Specialist,Male,1
5699,Malta,Database Coordinator,Male,1
5700,Malta,Database Coordinator,Female,1


In [59]:
# Now let's aggregate 

In [60]:
df_4['Quantity'] = 1

In [61]:
final_df = df_4[['country','title', 'gender', 'Quantity']].groupby(['country','title', 'gender'], as_index = False).count()

In [64]:
final_df['Percentage'] = (final_df['Quantity'] / final_df['Quantity'].sum()*100).round(2).astype(str) + '%'
final_df.sort_values(by='Quantity', ascending=False, inplace=True)

In [67]:
final_df

,country,title,gender,Quantity,Percentage
927,Germany,Oracle Database Administrator (Oracle DBA),Male,13,0.23%
531,France,Computer or Data Processing Systems Consultant,Male,13,0.23%
738,Germany,Data Architect,Male,12,0.21%
904,Germany,Geographic Information Systems Data Specialist...,Male,11,0.19%
1308,Italy,Database Administration Manager,Male,11,0.19%
...,...,...,...,...,...
80,Belgium,Data Management Associate,Female,1,0.02%
79,Belgium,Data Entry Specialist,Male,1,0.02%
78,Belgium,Data Entry Representative,Male,1,0.02%
77,Belgium,Data Entry Associate,Female,1,0.02%


In [68]:
final_df.to_csv(r'data/results/final_table.csv', index=False, header=True)